In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
import random
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

In [2]:
articles = pd.read_csv('../dataset/test_articles_dataset_newyork.csv', delimiter=',', keep_default_na=False)
tables = pd.read_csv('../dataset/data_tables_all_signal_newyork.csv', delimiter=',', keep_default_na=False)

In [3]:
embedding_model = Doc2Vec.load('../../pre_trained_models/doc2vec.bin')

/home/lss9/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [7]:
tables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52426 entries, 0 to 52425
Data columns (total 13 columns):
table_body                     52426 non-null object
table_caption                  52426 non-null object
table_domain_score             52426 non-null float64
table_header                   52426 non-null object
table_page_content             52426 non-null object
table_page_full_text           52426 non-null object
table_page_keywords            52426 non-null object
table_page_meta_description    52426 non-null object
table_page_meta_keywords       52426 non-null object
table_page_summary             52426 non-null object
table_page_tags                52426 non-null object
table_page_title               52426 non-null object
table_page_url                 52426 non-null object
dtypes: float64(1), object(12)
memory usage: 5.2+ MB


In [4]:
tables_vector = []
tables_ids = []

for i, row in tqdm(tables.iterrows()):
    
    table_title = row['table_page_title']
    table_page_cotent = row['table_page_content'][0:1000]
#     table_page_keywords = row['table_page_keywords']
    
    vector_words = tknzr.tokenize(table_title+" "+table_page_cotent)
    word_embedding = embedding_model.infer_vector(vector_words)
    
    tables_ids.append(row['table_page_url'])
    tables_vector.append(word_embedding) 

52426it [02:59, 292.46it/s]


In [5]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 10 columns):
article_full_text           148 non-null object
article_key_match           148 non-null object
article_keywords            148 non-null object
article_main_passage        148 non-null object
article_meta_description    148 non-null object
article_meta_keywords       148 non-null object
article_summary             148 non-null object
article_tags                148 non-null object
article_title               148 non-null object
article_url                 148 non-null object
dtypes: object(10)
memory usage: 11.6+ KB


In [6]:
articles_ids = []
articles_vector = []

for i, row in tqdm(articles.iterrows()):
    
    article_page_title = row['article_title']
    article_main_passage = row['article_main_passage'][0:1000]
#     article_keywords = row['keywords']
         
    #embedding
    vector_words = tknzr.tokenize(article_page_title+" "+article_main_passage)
    word_embedding = embedding_model.infer_vector(vector_words)
    
    articles_ids.append(row['article_key_match'])
    articles_vector.append(word_embedding)

148it [00:00, 166.78it/s]


In [7]:
def getAccuracy(idRankedTables, idQueryGoal):

    accuracy = 0

    for idTable in idRankedTables:
        
        if idTable[0] == idQueryGoal:
    
            accuracy = 1
            break;

    return accuracy

In [8]:
def saveAccuracy(k,accuracy):
    
    if k == 1:
            
        AverageTop1.append(accuracy)
    
    if k == 5:
            
        AverageTop5.append(accuracy)
        
    if k == 10:
            
        AverageTop10.append(accuracy)
        
    if k == 20:
            
        AverageTop20.append(accuracy)

In [9]:
AverageTop1 = []
AverageTop5 = []
AverageTop10 = []
AverageTop20 = []

topK = [1,5,10,20]

for i in tqdm(range(len(articles_vector))):
    
    idQueryGoal = articles_ids[i]
    
    distance_vector = pairwise_distances(articles_vector[i].reshape(1,300), tables_vector, metric='cosine')
    
    #creating the dataframe
    all_tables_score = []
    
    for j in range(len(tables_ids)):
        
        table_id = tables_ids[j]
        table_score = distance_vector[0][j]
        
        new_row = {"table_id": table_id,"table_score": table_score}
        
        all_tables_score.append(new_row)
        
    df_all_tables_scores = pd.DataFrame(all_tables_score)
    df_tables_sorting = df_all_tables_scores.sort_values('table_score')
    
    #compute the accuracy
    for accuracyK in topK:
        
        selected_top = df_tables_sorting.head(accuracyK)
        #min_score = selected_top['table_score'].max()
        #draw_tables_socres = df_tables_sorting[df_tables_sorting['table_score'] <= min_score]
        final_ranked_tables = selected_top.iloc[:,0:1].values
        
        accuracy_value = getAccuracy(final_ranked_tables,idQueryGoal)
        
        #save the accuracy on the list
        saveAccuracy(accuracyK,accuracy_value)

100%|██████████| 148/148 [00:27<00:00,  5.42it/s]


In [10]:
print(str(round(np.mean(AverageTop1),4)))
print(str(round(np.mean(AverageTop5),4)))
print(str(round(np.mean(AverageTop10),4)))
print(str(round(np.mean(AverageTop20),4)))

0.1622
0.3514
0.4257
0.527
